In [ ]:
import sys
sys.dont_write_bytecode = True

import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas

import h3
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as cx


# Local imports.
from resale_flat_prices.resale_flat_data.resale_flat_data import ResaleFlatData


# Data directories.
csv_data_dir = Path("../data/ResaleFlatPrices/")
processed_data_dir = Path("../data/processed_data/")

In [ ]:
# Load processed and geocoded data.
csv_file = "resale-flat-prices.csv.zip"

disp_columns = [
    "year_month", "town", "block", "street_name", "flat_type", "storey_range", "floor_area_sqm", "age",
    "resale_price", "price_per_sqm", "price_per_sqft"
]

resale_flat_data = ResaleFlatData(processed_data_dir / csv_file)
resale_flat_data.read_csv()
resale_flat_data.df = resale_flat_data.df.sort_values(["year_month", "town"])

resale_flat_data.df["quarter"] = resale_flat_data.df["month"].apply(lambda x: int(np.ceil(x / 3)))
resale_flat_data.df["year_quarter"] = resale_flat_data.df.apply(lambda DF: str(DF["year"]) + "-" + str(DF["quarter"]), axis = 1)

#display(resale_flat_data.df[disp_columns].tail())
print(resale_flat_data.df.shape)

In [ ]:
datetime_column = "year_quarter"
wanted_datetimes = [
    "2024-4", "2024-3", 
    #"2024-2", "2024-1",
]
price_column = "price_per_sqft"

resale_flat_data.df = resale_flat_data.df[resale_flat_data.df["year_quarter"].isin(wanted_datetimes)]
resale_flat_data.make_point_geometries(crs = "EPSG:4326")

In [ ]:
fig, ax = plt.subplots(figsize = [12, 12])
divider = make_axes_locatable(ax)
cax = divider.append_axes(**{"position": "right", "size": "5%", "pad": 0.1})

ax.set_xlim([103.60152080468028, 104.0470051248534])
ax.set_ylim([1.2359029533199608, 1.4733321131970046])
ax.set_title("{} {} {}.".format(datetime_column, wanted_datetimes, price_column))

resale_flat_data.df.plot(
    ax = ax,
    alpha = 1.0,
    column = price_column, 
    categorical = False,
    legend = True, 
    legend_kwds = {"label": "price_per_sqft"},
    cmap = 'viridis',
    edgecolor = None,
    cax = cax,
)

cx.add_basemap(ax, crs = resale_flat_data.df.crs, source = cx.providers.CartoDB.Positron)
fig.tight_layout()
plt.show()